## Libraries

In [1]:
# Standard libraries
import itertools as it

# Optional libraries for dev purposes
import os
import sys

import nibabel as nib
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import time
# Python implementation of Matlab's imresize due to differing results
# Original code found here: https://github.com/fatheral/matlab_imresize
# Note this code was last updated Aug. 2021
USER="ataha" # Update this to or path below to where imresize.py is located
sys.path.append(f'/home/ROBARTS/ataha/Desktop/auto-afids/autofid_main/training/')
from imresize import *

## Functions

In [2]:
def read_nii_metadata(nii_path):
    """Load in nifti data and header information and normalize MRI volume"""
    nii = nib.load(nii_path)
    nii_affine = nii.affine
    nii_data = nii.get_fdata()
    #normalization 
    nii_data = (nii_data - nii_data.min())/ (nii_data.max() - nii_data.min())
    return nii_affine, nii_data

In [3]:
def get_fid(fcsv_path, fid_num):
    """Extract specific fiducial's spatial coordinates"""
    fcsv_df = pd.read_csv(fcsv_path, sep=",", header=2)

    return fcsv_df.loc[fid_num, ["x", "y", "z"]].to_numpy(dtype="single")

In [4]:
def fid_voxel2world(fid_voxel, nii_affine, resample_size=1, padding=None):
    """Transform fiducials in voxel coordinates to world coordinates

    Optionally, resample to match resampled image
    """

    # Translation
    fid_world = fid_voxel.T + nii_affine[:3, 3:4]
    # Rotation
    fid_world = np.dot(fid_voxel, nii_affine[:3, :3])

    if padding:
        fid_voxel = np.pad(fid_voxel, padding, mode="constant")

    return fid_world.astype(float)

In [5]:
def image_resize(img, size):
    """Resize image using Matlab's imresize"""
    return imresize(img, size)

In [6]:
def integral_volume(resampled_image):
    '''Compute zero-padded resampled volume'''
    iv_image = resampled_image.cumsum(0).cumsum(1).cumsum(2)
    iv_zeropad = np.zeros((iv_image.shape[0]+1, iv_image.shape[1]+1, iv_image.shape[2]+1))
    iv_zeropad[1:, 1:, 1:] = iv_image
    
    return iv_zeropad

In [7]:
def fid_world2voxel(fid_world, nii_affine, resample_size=1, padding=None):
    """Transform fiducials in world coordinates to voxel coordinates

    Optionally, resample to match resampled image
    """

    # Translation
    fid_voxel = fid_world.T - nii_affine[:3, 3:4]
    # Rotation
    fid_voxel = np.dot(fid_voxel, np.linalg.inv(nii_affine[:3, :3]))

    # Round to nearest voxel
    fid_voxel = np.rint(np.diag(fid_voxel) * resample_size)

    if padding:
        fid_voxel = np.pad(fid_voxel, padding, mode="constant")

    return fid_voxel.astype(int)

In [8]:
#write results into fcsv file 
def seg_to_fcsv(weighted_centroids, fcsv_template, fcsv_output):
    # Read in fcsv template
    with open(fcsv_template, "r") as f:
        fcsv = [line.strip() for line in f]

    # Loop over fiducials
    for fid in range(1, 33):
        # Update fcsv, skipping header
        line_idx = fid + 2
        centroid_idx = fid - 1
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_x", str(weighted_centroids[centroid_idx][0])
        )
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_y", str(weighted_centroids[centroid_idx][1])
        )
        fcsv[line_idx] = fcsv[line_idx].replace(
            f"afid{fid}_z", str(weighted_centroids[centroid_idx][2])
        )

    # Write output fcsv
    with open(fcsv_output, "w") as f:
        f.write("\n".join(line for line in fcsv))

## Variables

In [9]:
sampling_space = 10
PAD_FLAG = False
PADDING = 0
#describes haar like feature coordiantes 
feature_offsets = f"/home/ROBARTS/{USER}/Desktop/auto-afids/autofid_main/training/feature_offsets.npz"
SIZE = 1  # Fixed for now but make variable

# Directories
DATASET = 'HCP'
BASE_DIR = f"/home/ROBARTS/{USER}/graham/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/autoafids"
T1W_DIR = f"{BASE_DIR}/data/{DATASET}/derivatives/afids_mni"
SUBJECT_IDS = [
    subject for subject in os.listdir(T1W_DIR) if "sub-" in subject]



In [10]:
start = time.time()

for sub in range(0,31): 
    SUBJECT = SUBJECT_IDS[sub]
    print(f'predicting on subject {SUBJECT}')
    arrays_of_afids = np.empty((3,), dtype=float)
    for i in range (1,33):
        FID_NUM = 30
        #MNI stuff 
        nii_path_mni = f"{T1W_DIR}/{SUBJECT}/{SUBJECT}_space-MNI152NLin2009cAsym_T1w.nii.gz"
        mni_aff, mni_img = read_nii_metadata(nii_path_mni)
        fcsv_path_mni = f"/home/ROBARTS/ataha/Desktop/autoafids/{SUBJECT}15.fcsv"
        print(f'predicting AFID#{FID_NUM}')
        #getting "consensus" coordinates of AFID in MNI to enable more efficinet training and less exhaustive search 
        fid_world_mni = get_fid(fcsv_path_mni, FID_NUM - 1)
        resampled_fid_mni = fid_world2voxel(fid_world_mni, mni_aff, resample_size=SIZE, padding=PADDING)

        # Load image -- assumes correct bids spec 
        nii_path = f"{T1W_DIR}/{SUBJECT}/{SUBJECT}_space-MNI152NLin2009cAsym_T1w.nii.gz"
        aff, img = read_nii_metadata(nii_path)
        # Resampled image
        resampled_image = image_resize(img, SIZE)
        # Optionally pad image
        if PADDING:
            resampled_image = np.pad(resampled_image, PADDING, mode="constant")

        # Get image samples intialized using MNI
        inner_its = [
            range(resampled_fid_mni[0] - sampling_space, resampled_fid_mni[0] + sampling_space+1),
            range(resampled_fid_mni[1] - sampling_space, resampled_fid_mni[1] + sampling_space+1),
            range(resampled_fid_mni[2] - sampling_space, resampled_fid_mni[2] + sampling_space+1),
        ]
        inner_samples = [t for t in it.product(*inner_its)]

        outer_its = [
            range(resampled_fid_mni[0] - sampling_space*2, resampled_fid_mni[0] + sampling_space*2+1, 2),
            range(resampled_fid_mni[1] - sampling_space*2, resampled_fid_mni[1] + sampling_space*2+1, 2),
            range(resampled_fid_mni[2] - sampling_space*2, resampled_fid_mni[2] + sampling_space*2+1, 2),
        ]
        outer_samples = [t for t in it.product(*outer_its)]

        # Concatenate and retain unique samples and
        all_samples = np.unique(
            np.array(inner_samples + outer_samples), axis=0)

        # Compute Haar-like features features
        # Make this optional to load or create
        feature_offsets_data = np.load(feature_offsets)
        smin, smax = feature_offsets_data["arr_0"], feature_offsets_data["arr_1"]

        # Generate bounding cube surrounding features
        min_corner_list = np.zeros((4000 * all_samples.shape[0], 3)).astype('uint8')
        max_corner_list = np.zeros((4000 * all_samples.shape[0], 3)).astype('uint8')

        for idx in range(all_samples.shape[0]):
            min_corner_list[idx*4000 : (idx+1)*4000] = all_samples[idx] + smin
            max_corner_list[idx*4000 : (idx+1)*4000] = all_samples[idx] + smax

        corner_list = np.hstack((min_corner_list, max_corner_list))

        #compute the integral image for more efficient generation of haar-like features
        iv_image = integral_volume(resampled_image)

        #intialize a numpy array to store features
        testerarr = np.zeros((4000 * all_samples.shape[0])) 

        # Generate features using integral volume
        numerator = (
            iv_image[corner_list[:, 3] + 1, corner_list[:, 4] + 1, corner_list[:, 5] + 1]
            - iv_image[corner_list[:, 0], corner_list[:, 4] + 1, corner_list[:, 5] + 1]
            - iv_image[corner_list[:, 3] + 1, corner_list[:, 4] + 1, corner_list[:, 2]]
            - iv_image[corner_list[:, 3] + 1, corner_list[:, 1], corner_list[:, 5] + 1]
            + iv_image[corner_list[:, 3] + 1, corner_list[:, 1], corner_list[:, 2]]
            + iv_image[corner_list[:, 0], corner_list[:, 1], corner_list[:, 5] + 1]
            + iv_image[corner_list[:, 0], corner_list[:, 4] + 1, corner_list[:, 2]]
            - iv_image[corner_list[:, 0], corner_list[:, 1], corner_list[:, 2]]
        )

        denomerator = (
            (corner_list[:, 3]-corner_list[:, 0]+1)
            *(corner_list[:, 4]-corner_list[:, 1]+1)
            *(corner_list[:, 5]-corner_list[:, 2]+1)
        )

        #dump features into intialized variable
        testerarr = numerator/denomerator
        vector1arr = np.zeros((4000 * all_samples.shape[0]))
        vector2arr = np.zeros((4000 * all_samples.shape[0]))

        for index in range(all_samples.shape[0]):
            vector = range(index * 4000, index * 4000 + 2000)
            vector1arr[index * 4000 : (index + 1) * 4000 - 2000] = vector

        for index in range(all_samples.shape[0]):
            vector = range(index * 4000 + 2000, index * 4000 + 4000)
            vector2arr[index * 4000 + 2000 : (index + 1) * 4000] = vector

        vector1arr[0] = 1
        vector1arr = vector1arr[vector1arr != 0]
        vector1arr[0] = 0
        vector2arr = vector2arr[vector2arr != 0]
        vector1arr = vector1arr.astype(int)
        vector2arr = vector2arr.astype(int)

        diff = testerarr[vector1arr] - testerarr[vector2arr]
        diff = np.reshape(diff, (all_samples.shape[0], 2000))


        #model prediction 
        clf = load(f'{FID_NUM}_{sampling_space}x{sampling_space}x{sampling_space}.joblib') 
        answer = clf.predict(diff)

        #extracting the smallest eucleidean distance from predictions 
        df = pd.DataFrame(answer)
        print(df[0].max())
        idx = df[0].idxmax()

        #reverse look-up to find voxel lowest distance coressponds to! 
        print(f'voxel coordinates with most liklihood of being AFID #{FID_NUM} are: {all_samples[idx]}')

        AFID = aff[:3, :3].dot(all_samples[idx]) + aff[:3, 3]
        print(f'coordinates in world are: {AFID}')

        arrays_of_afids = np.vstack((arrays_of_afids, AFID))

    seg_to_fcsv(arrays_of_afids[1:].astype(int), '/home/ROBARTS/ataha/Desktop/afids-auto/afids-auto-apply/resources/afids_template.fcsv', f'/home/ROBARTS/ataha/Desktop/autoafids/new_nativetrained/{SUBJECT}_{sampling_rate}.fcsv')

end = time.time()
print(f'time taken to predict all afids for all subject {end - start}')

predicting on subject sub-130013
predicting AFID#30
29.428035988539538
voxel coordinates with most liklihood of being AFID #30 are: [93 69 73]
coordinates in world are: [ -3. -63.  -5.]
predicting AFID#30


KeyboardInterrupt: 